In [1]:
# 1.对文档进行分词,剔除关键词
# 2.生成一个长度为9的滑动窗口，找到每个词语距离5以内的关联词语
# 3.得到票数最多的词语作为关联词

In [3]:
import jieba
import numpy as np
import jieba.posseg as posseg    #  词性标注

In [4]:
with open('./doc1.txt', 'r', encoding='utf-8') as file:
    text = file.read().replace('\n', '')
    

In [47]:
with open('../stopwords/中文停用词.txt', 'r', encoding='utf-8') as f:
    stopwords = list(map(lambda x: x.strip(), f.readlines()))
    
def cut_content(str_):
    return list(filter(
        lambda x: x not in stopwords and x.strip(),
        jieba.lcut(str_)
    ))

# cut_result = cut_content(text)

## graph 抽取关键词 

In [2]:
class WordGraph:
    def __init__(self, sentence, window, alpha, iternum):
        self.sentence = sentence
        self.window = window    # 窗口
        self.alpha = alpha,
        self.edge_dict = {}    # 记录节点的边链接字典
        self.iternum = iternum   # 迭代次数
        
    def cut_sentence(self, stopwords: list):
        """ 对句子分词，只保留形容词、动词、名词等 """
        tag_filter = ['a', 'd', 'n', 'v']
        cut_result = posseg.cut(self.sentence)
        # print(cut_result)
        self.word_list = [w.word for w in cut_result if w.flag in tag_filter]
        self.word_list = [w for w in self.word_list if w.strip() not in set(stopwords)]
        print(self.word_list)
        
    def create_notes(self):
        """ 根据窗口，构建每个节点的相邻节点，返回集合边 """
        # temp_list = []
        word_list_len = len(self.word_list)
        for index, word in enumerate(self.word_list):
            # 这里只考虑了第一次出现词语的情况，一个词语出现多次并未考虑
            temp_set = set()
                
            left = index - self.window + 1
            right = index + self.window
            # 窗口左右边界超出边界是设置为0和列表的最大长度
            if left < 0:
                left = 0
            if right > word_list_len:
                right = word_list_len
                
                # temp_list.append(word)
            for i in range(left, right):
                # 当前词语是他本身的时候跳过
                if i == index:
                    continue
                # 向集合中添加窗口范围的词语     
                temp_set.add(self.word_list[i])
            
            if word not in self.edge_dict:     
                self.edge_dict[word] = temp_set
            else:
                self.edge_dict[word] = self.edge_dict[word] | temp_set 
                
    def create_matrix(self):
        """ 构建矩阵，二维矩阵，一维表示关键词的边缘，第二维表示关键词，
        数值表示边缘词汇与中心词的关系程度
        """
        self.matrix = np.zeros((len(self.word_list), len(set(self.word_list))))
        self.word_index = {}
        self.word_dict = {}
        
        for i, v in enumerate(set(self.word_list)):
            self.word_index[v] = i
            self.word_dict[i] = v
            
        for key in self.edge_dict:
            for w in self.edge_dict[key]:
                self.matrix[self.word_index[key]][self.word_index[w]] = 1
                self.matrix[self.word_index[w]][self.word_index[key]] = 1
        
#         for j in range(self.matrix.shape[1]):
#             sum_ = 0
#             for i in range(self.matrix.shape[0]):
#                 sum_ += self.matrix[i][j]
#             for i in range(self.matrix.shape[0]):
#                 self.matrix[i][j] /= sum_
        
    def keywords_by_vote(self, nums=5):
        """ 通过投票的方式提取关键词 默认五个 """
        # 得到关键词的相关词的数量
        words_len_dict = {key: len(val) for key, val in self.edge_dict.items()}
        keywords_list = sorted(words_len_dict.items(), key=lambda x:x[1], reverse=True)[:nums]
        return [k[0] for k in keywords_list]

In [76]:
wg = WordGraph(text, 5, 0.5, 10)
wg.cut_sentence(stopwords)

['忙', '想要', '休息', '度假', '想到', '穷', '渴望', '富有', '怕', '幸福', '不能', '长久', '决定', '担心', '后悔', '没有', '属于', '常常', '心存', '欲望', '握', '怀念', '未', '拥有', '轻松', '做', '亲戚', '不会', '做', '朋友', '我会', '想起', '认识', '想起', '发生', '年老', '是否', '坐在', '互诉', '心声', '希望', '永远', '真诚', '相对', '朋友', '知己', '朋友', '喜欢', '了解', '愿', '珍惜', '朋友', '缘份', '成为', '朋友', '成为', '知己', '更', '难得', '时间', '未必', '成为', '知己', '原因', '一定', '证明', '朋友', '白费', '希望', '永远', '系', '朋友', '朋友', '时想', '分享', '朋友', '发脾气', '朋友', '没钱', '开饭', '打救', '朋友', '闷得', '发慌', '一同', '发慌', '朋友', '甘愿', '功课', '抄', '一同', '出', '猫', '一同', '人罚', '朋友', '买', '手', '信', '想', '买', '朋友', '看见', '上线', '想要', '体会', '价值', '问', '失败', '学生', '想要', '体会', '价值', '问', '不幸', '早产', '母亲', '想要', '体会', '价值', '问', '周刊', '编辑', '想要', '体会', '小时', '价值', '问', '等待', '相聚', '恋人', '想要', '体会', '价值', '问', '错过', '火车', '旅人', '想要', '体会', '价值', '问', '想要', '体会', '价值', '问', '错失', '金牌', '运动员', '朋友', '感动', '事情', '想', '分享', '朋友', '抱头', '哭', '扶', '肩膀', '朋友', '面对', '人生', '挫折', '一直', '紧握', '双手', '能够', '看到', '缘份', '能够', '做', 

In [77]:
wg.create_notes()
wg.create_matrix()

In [70]:
wg.keywords_by_vote(5)

['朋友', '想要', '体会', '价值', '问']

## textrank抽取关键词